## Libraries
geopandas may take a while to install. it has to be in python 2.7. ask mateo if there's problem.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt
from geopandas.tools import overlay

## Input shapefiles. 
sc_file is source shapefile with attribute values; tg_file is target shapefile (boundary obtained from amenitiy clusterning, in our case)

In [4]:
sc_file="/Users/June/urbanDNA/Ele-June/June's Data/boundaryFromR/sc_boundary/sc_boundary.shp"
tg_file="/Users/June/urbanDNA/Ele-June/June's Data/boundaryFromR/tg_boundary/tg_boundary.shp"
attr_name ="M_W_H_I"

In [5]:
sc_shp = gpd.read_file(sc_file)
tg_shp = gpd.read_file(tg_file)
sc_shp.head() #M_W_H_I is medium house income attached from census data.

,NAME,GSS_COD,HECTARE,NONLD_A,ONS_INN,SUB_2009,SUB_2006,M_W_H_I,geometry
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,None,"POLYGON ((516401.6 160201.8, 516407.3 160210.5..."
1,Croydon,E09000008,8649.441,0.000,F,None,None,643,"POLYGON ((535009.2 159504.7, 535005.5 159502, ..."
2,Bromley,E09000006,15013.487,0.000,F,None,None,718,"POLYGON ((540373.6 157530.4, 540361.2 157551.9..."
3,Hounslow,E09000018,5658.541,60.755,F,None,None,638,"POLYGON ((521975.8 178100, 521967.7 178096.8, ..."
4,Ealing,E09000009,5554.428,0.000,F,None,None,642,"POLYGON ((510253.5 182881.6, 510249.9 182886, ..."


## Calculation

In [6]:
sc_values = pd.to_numeric(sc_shp[attr_name])
tg_zone_value=[]

In [ ]:
for i in range(0,len(tg_shp)):
  value=0 # temporary variable whose value gets overwritten with every i. 
  for j in range(0,len(sc_shp)):
    inters=overlay(sc_shp.iloc[[j]],tg_shp.iloc[[i]],how="intersection")
   
    if len(inters) is not 0:
      inters_ratio=inters.geometry.area/tg_shp.geometry.area[i]
      sc_attr_val=sc_values[j] #8 because column 8 is the attribute we need to extract
      inters_wt =inters_ratio*sc_attr_val ## weighted intersection
    else:
       inters_wt=0 #if not intersecting then sc[j]'s contribution is zero
    
    # increment tg_zone_value
    value = value + inters_wt
  
  
  tg_zone_value.append(value)

In [144]:
tg_zone_value [5] #"interpolated value returned. in order. hence can be attached back to dataframe.

0    615.0
dtype: float64

### attach the returned value back to the tg_shp
but test we ggot value for each tg zone first 

In [33]:
len(tg_zone_value) 
len(tg_shp)
## but not really necessary because if the length does not equal,
## can't attach the array to the dataframe anyways.

500

In [34]:
tg_zone_value #is a weird 

type(tg_zone_value[1])
#tg_zone_value


pandas.core.series.Series

In [36]:
tg_shp[attr_name] = tg_zone_value
tg_shp.head()

,OA11CD,POPDEN,AvPTAI2015,PTAL,PTAIHigh,PTAILow,geometry,M_W_H_I
0,E00023264,115.2,14.14550,3,18.49880,11.81340,"POLYGON ((527635.452 171810.058, 527495.957000...",0 737.0 dtype: float64
1,E00003359,36.7,8.62391,2,9.36629,8.01509,"POLYGON ((543581.5159999999 171132.486, 543704...",0 718.0 dtype: float64
2,E00023266,183.4,9.67230,2,11.86470,7.47992,"POLYGON ((526613.539 171839.165, 526546.84 171...",0 737.0 dtype: float64
3,E00020264,96.1,65.49460,6b,67.29880,63.62840,"POLYGON ((533563.5209999999 179695.146, 533351...",0 589.0 dtype: float64
4,E00023263,165.2,14.65720,3,14.65720,14.65720,"POLYGON ((527822 172020, 527979 171903.001, 52...",0 737.0 dtype: float64


# TESTING individual bits of code (Scratch Paper Starts Here)

## Explore geopandas file structure

In [9]:
sc_shp.M_W_H_I.head()

0    None
1     643
2     718
3     638
4     642
Name: M_W_H_I, dtype: object

In [28]:
type(sc_shp.iloc[[2]])

geopandas.geodataframe.GeoDataFrame

#### Calculate area of polygon

geopandaName.geometry.area[row number]

In [11]:
type(sc_shp.geometry.area[2])

numpy.float64

### geo-series?

To access the float value insde the panda.Series

In [142]:
type(tg_zone_value[1])
tg_zone_value[1]
## ok it doesn't really matter because it is still treated as 

0    718.0
dtype: float64

In [138]:
# test_arr=[]
value=0 # temporary variable whose value gets overwritten with every i. 
for j in range(0,len(sc_shp)):
    inters=overlay(sc_shp.iloc[[j]],tg_shp.iloc[[5]],how="intersection")

    if len(inters) is not 0:
      inters_ratio=inters.geometry.area/tg_shp.geometry.area[i]
      sc_attr_val=sc_values[j] 
      inters_wt =inters_ratio*sc_attr_val ## weighted intersection
    else:
       inters_wt=0 #if not intersecting then sc[j]'s contribution is zero

    # increment tg_zone_value
    value += inters_wt 
    value.sum()


AttributeError: 'int' object has no attribute 'sum'

In [136]:
type(value)

pandas.core.series.Series

## Explore "overlap." And plotting. 

#### Intersection

In [112]:
#sc_shp.iloc[[2]]
inter2 = overlay(sc_shp.iloc[[2]] , tg_shp[1:2] , how="intersection")
# if(inter2 is 0):
#     print("null")
len(inter2) ## use len() is 0 or not 0 to check if there is overlap at all.

0    718.0
dtype: float64

In [56]:
# inters = overlay(sc_shp , tg_shp, how="intersection") 
# inters.dtypes

# inters.plot()
# plt.show(1) ## ALWAYS REMEMBER TO SHOW THE PLOT. OTHERWISE IT DOESNT SHOW....!!!!!

## Analogy to regular panda to familiarize with working with dataframe

In [133]:
### analogy to regular panda. basic stuff....

s = pd.DataFrame(np.random.randn(8, 4), columns=['A', 'B', 'C', 'D'])
s.A[2] ### dataframe.columnName[rowNumber]
s.iloc[1] = dict(A=1,B=2,C=1,D=0)
print(s.iloc[1]) # okay printed out vertically

A    1.0
B    2.0
C    1.0
D    0.0
Name: 1, dtype: float64


In [88]:
#s.head(1)
s[0:1] ### the way to slice / return the first row
s.iloc[[0]] ### this is a nicer way..

,A,B,C,D
0,0.214234,0.57994,0.399016,0.09169


In [135]:
pd.to_numeric(s['A'])

0    1.541759
1    1.000000
2   -1.085712
3    1.145229
4    0.573036
5   -0.011416
6   -0.226637
7   -1.364884
Name: A, dtype: float64

## Array (shamelessly...)

In [147]:
t=[]
t.append(1)
t.append(2)
t

[1, 2]